In [ ]:
!jupyter nbconvert socket.ipynb --to slides --post serve

[NbConvertApp] Converting notebook socket.ipynb to slides
[NbConvertApp] Writing 309969 bytes to socket.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/socket.slides.html
Use Control-C to stop this server


# Сети - 1

# man 2 socketpair

- почти как pipe

- но лучше, чем pipe

- можно общаться в обе стороны

```c
int socket[2];
socketpair(PF_LOCAL, SOCK_STREAM, 0, socket);  // ошибку нужно обработать

// CHILD PROCESS //

// пишем в одну сторону
char msg[] = "hello, parent";
write(socket[1], msg, sizeof(msg));

// читаем
char buffer[128];
int len = read(socket[1], buffer, sizeof(buffer));
write(1, buffer, len);
```

## shutdown

```c
shutdown(socket[1], SHUT_RDWR);
close(socket[1]);
```

# ip адрес
> без подробностей

## ip-v4

это 4 8-и битных чиселки

- 127.0.0.1

- 192.168.1.0

## ip-v6

это 6 8-и битных чиселок)

- c2:eb:d9:6b:68:ad

- 02:42:a3:a0:93:f9

- а ещё про него сегодня не будет)

# Port

## а зачем?

ip-адрес у нас один

да и то не совсем настоящий

а "общаться" с другими компами по сети тоже хочется

для решения таких проблем есть порты

## **порт** - это чиселка 16bit

Есть некоторые зарезервированные порты
- 20/21 FTP
- 22 SSH
- 25 SMTP (почта)
- 80 HTTP
- 443 HTTPS
- ... (их много)

первые 1024 порта открывать может только root (0 не используется, так что 1023)

1024 - 32767 можно открыть на подключение

32768 - 65535 (вроде) - порты для клиенских соединений; они открываются когда вы подключаетесь к какому-либо серверу

# Как подключиться к чему-нибудь?
> по TCP

нужно создать сокет

```c
// и обработать ошибку)
int socket_fd = socket(AF_INET, SOCK_STREAM, 0);
```

создать структурку с описанием

```c
struct sockaddr_in addr = {
    .sin_family = AF_INET,
    .sin_port = htons(8888),  // это чтоб перевести в BIG ENDIAN
    .sin_addr.s_addr = inet_addr("127.0.0.1"),  // это тоже
};
```

подключиться

```c
// ну и ошибку обработать
connect(socket_fd, (struct sockaddr*)(&addr), sizeof(addr)
```

потом можно начинать общаться

```c
// правда так делать не правильно)
write(socket_fd, "abc\n", sizeof("abc\n"));
// и так тоже
read(socket_fd, buffer, sizeof(buffer));
```

## а почему так не правильно?

```bash
$ man 2 read
...
RETURN VALUE
        ...
       It is not  an  error  if this  number  is smaller
       than the number of bytes requested; 
...
```

```bash
$ man 2 write
...
RETURN VALUE
       ...
       Note  that  a  successful  write() may transfer fewer
       than count bytes.  Such partial writes can occur  for
       various  reasons;  for example, because there was in‐
       sufficient space on the disk device to write  all  of
       the  requested bytes, or because a blocked write() to
       a socket, pipe, or similar was interrupted by a  sig‐
       nal handler after it had transferred some, but before
       it  had  transferred  all of the requested bytes.  In
       the event of a partial write, the caller can make an‐
       other write() call to transfer the  remaining  bytes.
       The  subsequent  call  will  either  transfer further
       bytes or may result in an error (e.g., if the disk is
       now full).
...
```

## страшно, очень страшно... а что делать то?

(хотяб в тасках на web и pipe-ы) - читайте и пишите через циклы

```c
char buffer[] = ...;

int total = 100;
int sent = 0;
while (sent < total) {
    sent += write(fd, buffer + sent, total - sent);
}
```

## назад к сокетам

надо сигнализировать, что общение закончилось
```c
shutdown(socket_fd, SHUT_RDWR);
close(socket_fd);
```

# Сервер
> тоже TCP

Всё почти то же самое

```c
struct sockaddr_in addr = {
    .sin_family = AF_INET,
    .sin_port = htons(8888),        // хотим BIG ENDIAN
    .sin_addr.s_addr = INADDR_ANY,  // константа, чтоб принимать
                                    // любые входящие
                                    // соединения
};

// сопоставляем себя с параметрами
bind(socket_fd, (struct sockaddr*)(&addr), sizeof(addr));

// говорим, что начинаем слушать соединения
listen(socket_fd, SOMAXCONN);

// подключаем клиента (можно подключать сразу несколько)
int client_fd = accept(socket_fd, NULL, NULL);
```

# HTTP vs HTTPS (Diffie-Hellman)

# להתראות